In [90]:
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.svm import SVR
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error, mean_absolute_error


In [2]:
df = pd.read_csv('wqi_full.csv')

In [3]:
df

,Unnamed: 0,Date,Alky pH 4.5,Chloride Ion,Orthophospht,Sulphate SO4,Ammonia(N),TurbidityNTU,Temp Water,Cond @ 25C,...,BOD ATU,COD as O2,Nitrate-N,Nitrite-N,Hardness,SALinsitu,ALL_RUNS,Average_count,WQI,WQI clf
0,2,2012-01-05,181.490909,113.369841,0.504410,71.920000,1.184673,14.750000,7.390233,1087.367089,...,16.172447,140.811111,10.844410,0.068047,283.500000,14.195000,1.333333,1.333333,0.573738,3
1,5,2012-01-09,220.684211,142.301587,0.819167,886.400000,2.830066,9.246154,9.044861,1176.500000,...,7.575197,66.107143,10.654855,0.063292,307.000000,22.524549,45.000000,45.000000,56.197425,1
2,8,2012-01-16,216.928571,361.460000,0.476744,118.950000,1.355232,35.100000,4.559789,1490.131343,...,22.517364,147.395833,10.564346,0.068554,313.025758,23.973000,29.954545,28.863636,5.273402,3
3,9,2012-01-17,235.390244,162.814286,0.819191,213.333333,4.351483,16.451852,3.959510,1156.449383,...,17.923786,113.724561,9.471521,0.049162,347.000000,22.524549,12.125000,6.500000,31.841305,2
4,13,2012-01-23,224.803922,105.854167,0.460703,95.354545,1.112021,9.292308,6.882867,1002.506849,...,5.865691,32.183673,8.754667,0.061222,399.181818,30.526250,2.200000,1.400000,38.055600,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326,1933,2022-07-09,202.702330,290.161737,0.395029,287.018100,2.840500,10.138017,18.425000,1115.134319,...,7.531000,74.000000,8.194472,0.063292,313.025758,22.524549,25.342105,22.355263,51.555623,1
327,1955,2022-08-06,202.702330,174.000000,0.395029,287.018100,1.200429,10.138017,18.075000,1115.134319,...,4.310390,49.272727,8.194472,0.063292,313.025758,33.933750,55.027027,54.837838,49.640626,2
328,1976,2022-09-10,202.702330,290.161737,0.395029,287.018100,2.901286,10.138017,18.841667,1115.134319,...,5.391538,42.500000,8.194472,0.063292,313.025758,32.533333,34.545455,11.727273,62.415078,1
329,2012,2022-11-05,202.702330,290.161737,0.395029,287.018100,3.107000,10.138017,13.281250,1115.134319,...,5.100625,30.153846,8.194472,0.063292,313.025758,34.711667,2.000000,1.000000,64.799889,1


In [4]:
df["Date"] = pd.to_datetime(df["Date"])

In [6]:
df = df.drop(columns={"ALL_RUNS","Unnamed: 0"})

In [7]:
df.isnull().sum()

Date             0
Alky pH 4.5      0
Chloride Ion     0
Orthophospht     0
Sulphate SO4     0
Ammonia(N)       0
TurbidityNTU     0
Temp Water       0
Cond @ 25C       0
Oxygen Diss      0
pH               0
BOD ATU          0
COD as O2        0
Nitrate-N        0
Nitrite-N        0
Hardness         0
SALinsitu        0
Average_count    0
WQI              0
WQI clf          0
dtype: int64

In [8]:
# df['year'] = df.Date.dt.year
# df['month'] = df.Date.dt.month
# df['day'] = df.Date.dt.day
df.columns

Index(['Date', 'Alky pH 4.5', 'Chloride Ion', 'Orthophospht', 'Sulphate SO4',
       'Ammonia(N)', 'TurbidityNTU', 'Temp Water', 'Cond @ 25C', 'Oxygen Diss',
       'pH', 'BOD ATU', 'COD as O2', 'Nitrate-N', 'Nitrite-N', 'Hardness',
       'SALinsitu', 'Average_count', 'WQI', 'WQI clf'],
      dtype='object')

In [78]:
cols = ['Alky pH 4.5', 'Chloride Ion', 'Orthophospht', 'Sulphate SO4',
       'Ammonia(N)', 'TurbidityNTU', 'Temp Water', 'Cond @ 25C', 'Oxygen Diss',
       'pH', 'BOD ATU', 'COD as O2', 'Nitrate-N', 'Nitrite-N', 'Hardness',
       'SALinsitu']
# X = df[cols]
# y = pd.DataFrame(df['WQI clf'])

X = df[cols]

y_wqi = pd.DataFrame(df['WQI'])

y_clf = df['WQI clf']



In [79]:
def cross_validation(model, _X, _y, _cv=5):
      '''Function to perform 5 Folds Cross-Validation
       Parameters
       ----------
      model: Python Class, default=None
              This is the machine learning algorithm to be used for training.
      _X: array
           This is the matrix of features.
      _y: array
           This is the target variable.
      _cv: int, default=5
          Determines the number of folds for cross-validation.
       Returns
       -------
       The function returns a dictionary containing the metrics 'accuracy', 'precision',
       'recall', 'f1' for both training set and validation set.
      '''
      scoring = ['neg_mean_absolute_error']
      results = cross_validate(estimator=model,
                               X=_X,
                               y=_y,
                               cv=5,
                               scoring=scoring,
                               return_train_score=True,
                              )
      
      return {
          "test_mae_mean" : -1 *results['test_neg_mean_absolute_error'].mean(), 
          "test_mae_std" : results['test_neg_mean_absolute_error'].std()
          
      }

### Regression

In [80]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_wqi, test_size=0.2, random_state=42)


#### Support Vector Regression


In [102]:
# Create an instance of SVR and fit the training data
svr = SVR(kernel='linear', C=1.0)
svr.fit(X_train, y_train)

# Use the trained model to predict on the testing data
y_pred_svr = svr.predict(X_test)

# Evaluate the model using mean squared error
mse = mean_squared_error(y_test, y_pred_svr)
rmse = np.sqrt(mean_squared_error(y_test, y_pred_svr))
r2 = r2_score(y_test, y_pred_svr)
mae = mean_absolute_error(y_test, y_pred_svr)

print("RMSE:", rmse)
print("MSE:", mse)
print("MAE:", mae)
print('R-squared:', r2)

RMSE: 0.6056719517612318
MSE: 0.36683851315025995
MAE: 0.43902749675458863
R-squared: 0.4908045499284117


#### Linear regression 

In [101]:
# Create a linear regression model and fit the data
model = LinearRegression()
model.fit(X_train, y_train)


# Use the trained model to predict on the testing data
y_pred_ml = model.predict(X_test)


# Evaluate the model 
mse = mean_squared_error(y_test, y_pred_ml)
rmse = np.sqrt(mean_squared_error(y_test, y_pred_ml))
r2 = r2_score(y_test, y_pred_ml)
mae = mean_absolute_error(y_test, y_pred_ml)

print("RMSE:", rmse)
print("MSE:", mse)
print("MAE:", mae)
print('R-squared:', r2)

RMSE: 0.47625193872295013
MSE: 0.22681590913736863
MAE: 0.2980462751870514
R-squared: 0.6851649300811231


#### Decision Tress regression

In [96]:
# Create a decision tree regressor model and fit the training data
model = DecisionTreeRegressor(random_state=42)
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred_dt = model.predict(X_test)

# Evaluate the model using mean squared error
mse = mean_squared_error(y_test, y_pred_dt)
rmse = np.sqrt(mean_squared_error(y_test, y_pred_dt))
mse = mean_squared_error(y_test, y_pred_dt)
mae = mean_absolute_error(y_test, y_pred_dt)
r2 = r2_score(y_test, y_pred_dt)

print("RMSE:", rmse)
print("MSE:", mse)
print("MAE:", mae)
print('R-squared:', r2)

RMSE: 0.8375515132297433
MSE: 0.7014925373134329
MAE: 0.4626865671641791
R-squared: 0.026283240568954924


In [98]:
# Create a Random Forest  regression model and cross validate the data
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred_rf = model.predict(X_test)

# Evaluate the model using mean squared error
mse = mean_squared_error(y_test, y_pred_rf)
rmse = np.sqrt(mean_squared_error(y_test, y_pred_rf))
mse = mean_squared_error(y_test, y_pred_rf)
mae = mean_absolute_error(y_test, y_pred_rf)
r2 = r2_score(y_test, y_pred_rf)

print("RMSE:", rmse)
print("MSE:", mse)
print("MAE:", mae)
print('R-squared:', r2)


lg = LGBMRegressor(random_state=42)



RMSE: 0.5142071131595523
MSE: 0.2644089552238806
MAE: 0.3617910447761194
R-squared: 0.6329833642547928


In [99]:
# Create a LGBM regression model and cross validate the data
model = LGBMRegressor(random_state=42)
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred_lgbm = model.predict(X_test)

# Evaluate the model using mean squared error
mse = mean_squared_error(y_test, y_pred_lgbm)
rmse = np.sqrt(mean_squared_error(y_test, y_pred_lgbm))
mse = mean_squared_error(y_test, y_pred_lgbm)
mae = mean_absolute_error(y_test, y_pred_lgbm)
r2 = r2_score(y_test, y_pred_lgbm)

print("RMSE:", rmse)
print("MSE:", mse)
print("MAE:", mae)
print('R-squared:', r2)



RMSE: 0.5416582320907857
MSE: 0.29339364039171545
MAE: 0.3597759961238674
R-squared: 0.5927507570444


### Classification

In [103]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_clf, test_size=0.2, random_state=42)


#### Logistic Regression 

In [114]:
from sklearn.linear_model import LogisticRegression

# Create a Logistic Regression model object
logreg = LogisticRegression()

# Train the model using training data
logreg.fit(X_train, y_train)

# Predict the class labels for test data
y_pred_clg = logreg.predict(X_test)


# Evaluate the model using mean squared error
mse = mean_squared_error(y_test, y_pred_clg)
rmse = np.sqrt(mean_squared_error(y_test, y_pred_clg))
mse = mean_squared_error(y_test, y_pred_clg)
mae = mean_absolute_error(y_test, y_pred_clg)
r2 = r2_score(y_test, y_pred_clg)

print("RMSE:", rmse)
print("MSE:", mse)
print("MAE:", mae)
print('R-squared:', r2)

RMSE: 0.7431277184778867
MSE: 0.5522388059701493
MAE: 0.373134328358209
R-squared: 0.233457019171305


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


#### Decision Tree classifier

In [110]:
from sklearn.tree import DecisionTreeClassifier

# Create a Decision Tree Classifier object
dt = DecisionTreeClassifier()

# Train the model using training data
dt.fit(X_train, y_train)

# Predict the class labels for test data
y_pred_dtc = dt.predict(X_test)

# Evaluate the model using mean squared error
mse = mean_squared_error(y_test, y_pred_dtc)
rmse = np.sqrt(mean_squared_error(y_test, y_pred_dtc))
mse = mean_squared_error(y_test, y_pred_dtc)
mae = mean_absolute_error(y_test, y_pred_dtc)
r2 = r2_score(y_test, y_pred_dtc)

print("RMSE:", rmse)
print("MSE:", mse)
print("MAE:", mae)
print('R-squared:', r2)

RMSE: 0.8011186209146055
MSE: 0.6417910447761194
MAE: 0.43283582089552236
R-squared: 0.10915275200989494


#### Random Forest classifier 

In [111]:
from sklearn.ensemble import RandomForestClassifier

# Create a Random Forest Classifier object
rf = RandomForestClassifier()

# Train the model using training data
rf.fit(X_train, y_train)

# Predict the class labels for test data
y_pred_rfc = rf.predict(X_test)

# Evaluate the model using mean squared error
mse = mean_squared_error(y_test, y_pred_rfc)
rmse = np.sqrt(mean_squared_error(y_test, y_pred_rfc))
mse = mean_squared_error(y_test, y_pred_rfc)
mae = mean_absolute_error(y_test, y_pred_rfc)
r2 = r2_score(y_test, y_pred_rfc)

print("RMSE:", rmse)
print("MSE:", mse)
print("MAE:", mae)
print('R-squared:', r2)

RMSE: 0.5859040724295678
MSE: 0.34328358208955223
MAE: 0.2835820895522388
R-squared: 0.5235003092145949


#### support vector classification

In [112]:
from sklearn.svm import SVC

# Create an SVM object
svm = SVC()

# Train the model using training data
svm.fit(X_train, y_train)

# Predict the class labels for test data
y_pred_svc = svm.predict(X_test)


# Evaluate the model using mean squared error
mse = mean_squared_error(y_test, y_pred_svc)
rmse = np.sqrt(mean_squared_error(y_test, y_pred_svc))
mse = mean_squared_error(y_test, y_pred_svc)
mae = mean_absolute_error(y_test, y_pred_svc)
r2 = r2_score(y_test, y_pred_svc)

print("RMSE:", rmse)
print("MSE:", mse)
print("MAE:", mae)
print('R-squared:', r2)

RMSE: 0.8551861104941366
MSE: 0.7313432835820896
MAE: 0.6119402985074627
R-squared: -0.015151515151515138


#### Naive Bayes classification

In [113]:
from sklearn.naive_bayes import GaussianNB

# Create a Gaussian Naive Bayes object
nb = GaussianNB()

# Train the model using training data
nb.fit(X_train, y_train)

# Predict the class labels for test data
y_pred_nb = nb.predict(X_test)


# Evaluate the model using mean squared error
mse = mean_squared_error(y_test, y_pred_nb)
rmse = np.sqrt(mean_squared_error(y_test, y_pred_nb))
mse = mean_squared_error(y_test, y_pred_nb)
mae = mean_absolute_error(y_test, y_pred_nb)
r2 = r2_score(y_test, y_pred_nb)

print("RMSE:", rmse)
print("MSE:", mse)
print("MAE:", mae)
print('R-squared:', r2)

RMSE: 0.922359077939127
MSE: 0.8507462686567164
MAE: 0.5522388059701493
R-squared: -0.18089053803339517


#### Neural network

In [109]:
import tensorflow as tf

# Create a Sequential model object
model = tf.keras.Sequential()

# Add layers to the model
model.add(tf.keras.layers.Dense(units=64, activation='relu', input_dim=10))
model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model using training data
model.fit(X_train, y_train, epochs=10, batch_size=32)

# Predict the class labels for test data
y_pred = model.predict_classes(X_test)


2023-04-24 20:55:26.385492: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-24 20:55:26.595794: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-24 20:55:26.649268: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-04-24 20:55:27.764030: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural N

Epoch 1/10


ValueError: in user code:

    File "/usr/local/lib/python3.8/dist-packages/keras/engine/training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.8/dist-packages/keras/engine/training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.8/dist-packages/keras/engine/training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.8/dist-packages/keras/engine/training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "/usr/local/lib/python3.8/dist-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/usr/local/lib/python3.8/dist-packages/keras/engine/input_spec.py", line 295, in assert_input_compatibility
        raise ValueError(

    ValueError: Input 0 of layer "sequential" is 

In [76]:
# Separate the features and target variable
# X = data.drop('target', axis=1)
# y = data['target']
# !!pip install imblearn

from imblearn.over_sampling import ADASYN

from sklearn import preprocessing
from sklearn import utils

#convert y values to categorical values
lab = preprocessing.LabelEncoder()
y_transformed = lab.fit_transform(y_clf)
# Instantiate the ADASYN class
adasyn = ADASYN()

# Fit and transform the data
X_resampled, y_resampled = adasyn.fit_resample(X, y_clf)

# Print the number of samples before and after resampling
print("Number of samples before resampling:", len(X))
print("Number of samples after resampling:", len(X_resampled))

Number of samples before resampling: 331
Number of samples after resampling: 602


In [77]:
X_resampled.to_csv('resampled.csv')

In [68]:
X_resampled

,Alky pH 4.5,Chloride Ion,Orthophospht,Sulphate SO4,Ammonia(N),TurbidityNTU,Temp Water,Cond @ 25C,Oxygen Diss,pH,BOD ATU,COD as O2,Nitrate-N,Nitrite-N,Hardness,SALinsitu,WQI,WQI clf
0,181.490909,113.369841,0.504410,71.920000,1.184673,14.750000,7.390233,1087.367089,10.031494,7.860330,16.172447,140.811111,10.844410,0.068047,283.500000,14.195000,0.573738,3
1,220.684211,142.301587,0.819167,886.400000,2.830066,9.246154,9.044861,1176.500000,10.089863,7.884348,7.575197,66.107143,10.654855,0.063292,307.000000,22.524549,56.197425,1
2,216.928571,361.460000,0.476744,118.950000,1.355232,35.100000,4.559789,1490.131343,11.696173,7.743488,22.517364,147.395833,10.564346,0.068554,313.025758,23.973000,5.273402,3
3,235.390244,162.814286,0.819191,213.333333,4.351483,16.451852,3.959510,1156.449383,12.124239,7.839619,17.923786,113.724561,9.471521,0.049162,347.000000,22.524549,31.841305,2
4,224.803922,105.854167,0.460703,95.354545,1.112021,9.292308,6.882867,1002.506849,9.910886,7.890313,5.865691,32.183673,8.754667,0.061222,399.181818,30.526250,38.055600,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
597,192.572566,164.884623,0.078094,287.018100,1.728049,10.138017,9.964029,1627.458106,9.540230,7.653206,11.788106,114.291903,6.880556,0.027401,300.753510,26.231837,8.998514,3
598,192.849920,20.134155,0.089404,287.018100,1.124294,8.133829,14.536357,324.132946,10.012088,7.667425,10.439801,172.343685,7.071581,0.017464,318.772006,28.357869,6.381011,3
599,201.468153,240.401592,0.384434,1048.105366,2.886396,10.094952,10.130083,1117.035717,9.765357,7.819835,17.236249,182.671022,7.013061,0.061927,311.646505,25.438081,5.870344,3
600,181.895126,71.474484,0.330955,100.041733,3.770544,3.676516,11.977388,689.899998,11.461038,8.036840,16.554649,323.546080,4.754899,0.036291,324.100706,23.533076,4.084403,3
